In [1]:
import regex as re
import pandas as pd

In [2]:
#first game
sample_game = pd.read_excel('Outline_And_Sample.xlsx.xlsx',
                             sheet_name='Sample',
                             skiprows=1,
                             usecols=['TIME','TEAM','PLAY','SCORE','Prev State','New State'])

In [3]:
#second game
sample_game = pd.read_excel('Second_Sample_Game.xlsx')

In [4]:
#second sample obtained programatically and more clean than first
# sample_game['TEAM']=sample_game['TEAM'].str.replace(' title=','')

In [5]:
sample_game=sample_game.fillna('')

In [6]:
#first sample game
team_A = 'UT'
team_B = 'LSU'

#second sample game
team_A = 'SJSU'
team_B = 'Nevada'

In [7]:
new_states = []
for idx in range(len(sample_game)):
    new_state='new_state'
    current_play = sample_game['PLAY'][idx]
    # current_state = sample_game['Prev State'][idx]
    #logic for "UNNEC" rows
    if "MISSED" in current_play:
        new_state='UNNEC'
    elif "Steal" in current_play:
        new_state='UNNEC'
    elif 'Assist' in current_play:
        new_state='UNNEC'
    elif 'Block' in current_play:
        new_state='UNNEC'
    elif "Subbing" in current_play:
        new_state='UNNEC'
    elif "timeout" in current_play:
        new_state='UNNEC'
    elif "time out" in current_play:
        new_state = 'UNNEC'

    else:
        if team_A in current_play:

            #Logic for Fouls
            if 'Foul' in current_play:
                #if foul happens, we need the next play to determine what the result is
                #Will need to amend if the shot is GOOD before the foul?
                #Confirmed - if shot is GOOD before the foul, then foul results in UNNEC and previous line results in f2 or f3
                #Also need to check timestamp before foul - can have GOOD shot on previous separate play
                #may need to do same backwards iteration to find "valid plays"

                curr_time=sample_game['TIME'][idx]
                prev_play='Filler String'
                prev_time = 'Filler String'
                try:
                    prev_play = sample_game['PLAY'][idx-1]
                    prev_time = sample_game['TIME'][idx-1]
                except KeyError:
                    pass

                if ('GOOD' in prev_play) & (curr_time==prev_time):
                    new_state = 'UNNEC'

                #sadly, we need to account for subs and timeouts in between foul and FTs
                    
                else:
                    play_iterator=1
                    valid_play = False
                    while not valid_play:
                        next_play = sample_game['PLAY'][idx+play_iterator]
                        if ('Sub' in next_play) | ('timeout' in next_play) | ('time out' in next_play):
                            valid_play=False
                            play_iterator+=1
                        else:
                            valid_play=True

                    if 'Free Throw' in next_play:
                        new_state = 'Bf0'
                    else:
                        new_state = 'Bi0'

            #Logic for MADE free throws
            elif "Free Throw GOOD" in current_play:
                #if free throw is made, we need the next play to determine if next play is FT or inbound
                #sadly, we need to account for subs and timeouts in between FTs
                #NOTE - THIS WILL BE DIFFERENT WITH TECHNICAL FOULS

                play_iterator=1
                valid_play = False
                while not valid_play:
                    next_play = sample_game['PLAY'][idx+play_iterator]
                    if ('Sub' in next_play) | ('timeout' in next_play) | ('time out' in next_play):
                        valid_play=False
                        play_iterator+=1
                    else:
                        valid_play=True

                if 'Free Throw' in next_play:
                    new_state = 'Af1'
                else:
                    new_state = 'Bi1'

            #Logic for "REBOUND"
            elif "REBOUND" in current_play:
                #if rebound on DEAD BALL, this either transitions to same team FT OR non-shooting foul so inbound?
                #Try this - if previous play is "Free Throw MISSED" then transitions to same team FT. Else to inbound.
                try:
                    prev_play = sample_game['PLAY'][idx-1]
                except KeyError:
                    pass
                
                if 'dead ball' in current_play:
                    if "Free Throw MISSED" in prev_play:
                        new_state = 'Af0'
                    else:
                        new_state = 'Ai0'
                else:
                    new_state = 'Ar0'

            #Logic for "TURNOVER"
            elif "Turnover" in current_play:
                new_state='Br0'

            #Logic for "GOOD" shots
            elif "GOOD" in current_play:
                #determine point value
                pv=0
                if 'Jumper' in current_play:
                    pv=2
                elif '3 Pointer' in current_play:
                    pv=3
                elif 'Layup' in current_play:
                    pv=2
                elif 'Dunk' in current_play:
                    pv=2
                #Need to check next play for FOUL AND TIMESTAMP
                current_time = sample_game['TIME'][idx]

                next_play = sample_game['PLAY'][idx+1]
                next_time = sample_game['TIME'][idx+1]
                if ('Foul' in next_play) & (next_time == current_time):
                    new_state = 'Af{}'.format(str(pv))
                else:
                    new_state = 'Bi{}'.format(str(pv))

            else:
                new_state = 'Team A Logic Missed'




        elif team_B in current_play:

            #Logic for Fouls
            if 'Foul' in current_play:
                #if foul happens, we need the next play to determine what the result is
                #Will need to amend if the shot is GOOD before the foul?
                #Confirmed - if shot is GOOD before the foul, then foul results in UNNEC and previous line results in f2 or f3
                #Also need to check timestamp before foul - can have GOOD shot on previous separate play
                #may need to do same backwards iteration to find "valid plays"

                curr_time=sample_game['TIME'][idx]
                prev_play='Filler String'
                prev_time = 'Filler String'
                try:
                    prev_play = sample_game['PLAY'][idx-1]
                    prev_time = sample_game['TIME'][idx-1]
                except KeyError:
                    pass

                if ('GOOD' in prev_play)&(curr_time==prev_time):
                    new_state = 'UNNEC'

                #sadly, we need to account for subs and timeouts in between foul and FTs
                    
                else:
                    play_iterator=1
                    valid_play = False
                    while not valid_play:
                        next_play = sample_game['PLAY'][idx+play_iterator]
                        if ('Sub' in next_play) | ('timeout' in next_play) | ('time out' in next_play):
                            valid_play=False
                            play_iterator+=1
                        else:
                            valid_play=True

                    if 'Free Throw' in next_play:
                        new_state = 'Af0'
                    else:
                        new_state = 'Ai0'

            #Logic for MADE free throws
            elif "Free Throw GOOD" in current_play:
                #if free throw is made, we need the next play to determine if next play is FT or inbound
                #sadly, we need to account for subs and timeouts in between FTs
                #NOTE - THIS WILL BE DIFFERENT WITH TECHNICAL FOULS
                play_iterator=1
                valid_play = False
                while not valid_play:
                    next_play = sample_game['PLAY'][idx+play_iterator]
                    if ('Sub' in next_play) | ('timeout' in next_play) | ('time out' in next_play):
                        valid_play=False
                        play_iterator+=1
                    else:
                        valid_play=True

                if 'Free Throw' in next_play:
                    new_state = 'Bf1'
                else:
                    new_state = 'Ai1'

            #Logic for "REBOUND"
            elif "REBOUND" in current_play:
                #if rebound on DEAD BALL, this either transitions to same team FT OR non-shooting foul so inbound?
                #Try this - if previous play is "Free Throw MISSED" then transitions to same team FT. Else to inbound.
                try:
                    prev_play = sample_game['PLAY'][idx-1]
                except KeyError:
                    pass
                
                if 'dead ball' in current_play:
                    if "Free Throw MISSED" in prev_play:
                        new_state = 'Bf0'
                    else:
                        new_state = 'Bi0'
                else:
                    new_state = 'Br0'

            #Logic for "TURNOVER"
            elif "Turnover" in current_play:
                new_state='Ar0'

            #Logic for "GOOD" shots
            elif "GOOD" in current_play:
                #determine point value
                pv=0
                if 'Jumper' in current_play:
                    pv=2
                elif '3 Pointer' in current_play:
                    pv=3
                elif 'Layup' in current_play:
                    pv=2
                elif 'Dunk' in current_play:
                    pv=2
                #Need to check next play for FOUL AND TIMESTAMP (AND ASSIST AND SUBS :(    )
                #Possible soln- if GOOD results in Foul, make GOOD unnecessary and make Foul end with FT?
                    #or maybe simplify by checking for future FOUL and if there is a future FOUL at same or nearly same timestamp, just allow it to be an inbound-2 and then FT-0 
                    #(this is clearly a mirepresentation of what happens but that is okay for now)
                current_time = sample_game['TIME'][idx]

                next_play = sample_game['PLAY'][idx+1]
                next_time = sample_game['TIME'][idx+1]
                if ('Foul' in next_play) & (next_time == current_time):
                    new_state = 'Bf{}'.format(str(pv))
                else:
                    new_state = 'Ai{}'.format(str(pv))

            else:
                new_state = 'Team B logic missed'
        
        else:
            new_state = "all logic missed"
    new_states.append(new_state)

In [8]:
new_states

['UNNEC',
 'Br0',
 'Bi0',
 'Ai3',
 'UNNEC',
 'Bi3',
 'UNNEC',
 'Ai2',
 'Bi3',
 'Ai3',
 'UNNEC',
 'Br0',
 'UNNEC',
 'Br0',
 'Bi0',
 'UNNEC',
 'UNNEC',
 'Ai2',
 'UNNEC',
 'UNNEC',
 'Br0',
 'Ar0',
 'UNNEC',
 'UNNEC',
 'Br0',
 'UNNEC',
 'Br0',
 'Bf0',
 'UNNEC',
 'Bf0',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'Ai1',
 'UNNEC',
 'Ar0',
 'UNNEC',
 'Ar0',
 'UNNEC',
 'Br0',
 'UNNEC',
 'Br0',
 'UNNEC',
 'Ai3',
 'UNNEC',
 'Br0',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'Ar0',
 'Ai0',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'Bi2',
 'Ar0',
 'UNNEC',
 'Br0',
 'UNNEC',
 'Ai3',
 'UNNEC',
 'Bf0',
 'UNNEC',
 'UNNEC',
 'Ai1',
 'UNNEC',
 'UNNEC',
 'Bi3',
 'UNNEC',
 'UNNEC',
 'Br0',
 'Bi0',
 'UNNEC',
 'UNNEC',
 'Ai2',
 'UNNEC',
 'UNNEC',
 'Br0',
 'Bf0',
 'Bf1',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'Ai1',
 'UNNEC',
 'UNNEC',
 'Ai0',
 'UNNEC',
 'Br0',
 'Bf0',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'Bf0',
 'Ai1',
 'UNNEC',
 'Br0',
 'UNNEC',
 'Ar0',
 'Ai0',
 'UNNEC',
 'Br0',
 'Bf0',


In [7]:
#TRYING TO USE HELPER FUNCTION VERSION
from helper_functions import game_states_logic

In [8]:
states_attempt = game_states_logic.get_play_states(sample_game, team_A, team_B)

In [9]:
states_attempt

['UNNEC',
 'Br0',
 'Bi0',
 'Ai3',
 'UNNEC',
 'Bi3',
 'UNNEC',
 'Ai2',
 'Bi3',
 'Ai3',
 'UNNEC',
 'Br0',
 'UNNEC',
 'Br0',
 'Bi0',
 'UNNEC',
 'UNNEC',
 'Ai2',
 'UNNEC',
 'UNNEC',
 'Br0',
 'Ar0',
 'UNNEC',
 'UNNEC',
 'Br0',
 'UNNEC',
 'Br0',
 'Bf0',
 'UNNEC',
 'Bf0',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'Ai1',
 'UNNEC',
 'Ar0',
 'UNNEC',
 'Ar0',
 'UNNEC',
 'Br0',
 'UNNEC',
 'Br0',
 'UNNEC',
 'Ai3',
 'UNNEC',
 'Br0',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'Ar0',
 'Ai0',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'Bi2',
 'Ar0',
 'UNNEC',
 'Br0',
 'UNNEC',
 'Bf3',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'Ai1',
 'UNNEC',
 'UNNEC',
 'Bi3',
 'UNNEC',
 'UNNEC',
 'Br0',
 'Bi0',
 'UNNEC',
 'UNNEC',
 'Ai2',
 'UNNEC',
 'UNNEC',
 'Br0',
 'Bf0',
 'Bf1',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'Ai1',
 'UNNEC',
 'UNNEC',
 'Ai0',
 'UNNEC',
 'Br0',
 'Bf0',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'Bf0',
 'Ai1',
 'UNNEC',
 'Br0',
 'UNNEC',
 'Ar0',
 'Ai0',
 'UNNEC',
 'Br0',
 'Bf0'